# Export Table Schema Information as a HTML

In [ ]:
# Install the required libraries for newly created environment - only once

#!pip install mysql-connector-python
#!pip install pandas

In [ ]:
# Import required libraries
import os
import mysql.connector
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
from mysql.connector import connect, Error

In [ ]:
# Configure username, password and database details
db_user = "<<user_name>>"
db_password = "<<user_password""
bd_host = "<<host_name>>"
db_port = "<<port>>"
db_name = "<<database_name>>"

In [ ]:
# Function to establish a database connection
def _get_db_connection(db_host, db_port, db_user, db_password, db_name):
    """
    Establish a database connection and return the connection object.

    Args:
        db_host (str): The host address of the database server.
        db_port (int): The port number for the database server.
        db_user (str): The username for connecting to the database.
        db_password (str): The password for the database user.
        db_name (str): The name of the database to connect to.

    Returns:
        Connection: A database connection object.

    Returns None if an error occurs during connection.
    """
    try:
        cnx = connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_password, 
            database=db_name)
        return cnx
    except Error as e:
        print(f"Error: {e}")
        return None

In [ ]:
# Uncomment to validate database connection issue

#cnx = _get_db_connection(db_host, db_port, db_user, db_password, db_name)
#if cnx is not None:
#    print("Database connection - successful")
#else:
#    print("Database connection - failure")

In [ ]:
# Function to get html template
def _get_html_template():
    """
    :return HTML Template
    """
    _html_template = '''
<!DOCTYPE html>
<html lang="en">
    <head>
        <title>Database Schema Tables Information</title>
        <style>
            .dataframe {{
                font-size: 11pt; 
                font-family: Arial;
                border-collapse: collapse; 
                border: 1px solid silver;
            }}
            .dataframe thead > tr {{
                background: #c1cfff;
            }}
            .dataframe td, th {{
                padding: 5px;
            }}
            .dataframe tr:nth-child(even) {{
                background: #e9f0ff;
            }}
            .dataframe tr:hover {{
                background: #83afff;
                cursor: pointer;
            }}
        </style>
    </head>
    <body>
        {tables}        
    </body>
</html>
'''
    return _html_template

In [ ]:
def _get_table_div_template():
    """
    :return HTML Table Div Template
    """
    _table_div_template = '''
<div class="sqltable">
    <h2>
        {tablename}
    </h2>
    {table}        
</div>
'''
    return _table_div_template

# Generate html 

In [ ]:
def _create_file(html):
    file_name=f"generated/{db_name}_schema_information.html" 
    if not os.path.exists("generated"):
        os.makedirs("generated")
    with open(file_name, 'w') as f:
        f.write(BeautifulSoup(html, 'html.parser').prettify())

In [ ]:
def _export_table_schema_as_html(db_host, db_port, db_user, db_password, db_name):
    """
    """
    show_db_query = "SHOW TABLES"
    try:
        cnx = _get_db_connection(db_host, db_port, db_user, db_password, db_name)
        with cnx.cursor(buffered=True) as cursor:
            cursor.execute(show_db_query)
            tables = cursor.fetchall()
            html_tables={}
            for table in tables:
                #print(table)
                table_name = table[0]           
                describe_table=f'DESCRIBE {table_name}'
                cursor.execute(describe_table)
                column_rows = cursor.fetchall()            
                schemas=[]
                for column_row in column_rows:
                    #print([str(x).lower() for x in column_row])
                    schemas.append([str(x).lower() for x in column_row])
                df = pd.DataFrame(schemas,columns=['Field','Type','Null','Key','Default','Extra'])
                df.index = df.index + 1
                html_table = df.to_html(justify="left")
                html_tables[table_name]=html_table
                #print(html_table)

            div_tables=''
            i=1
            for key in html_tables.keys():
                div_tables += _get_table_div_template().format(tablename="{0}. {1}".format(i,key), table=html_tables[key])
                i+=1
            
            # Create File
            _create_file(_get_html_template().format(tables=div_tables))

        cursor.close()
        cnx.close()
        return None
    except Error as e:
        return f"Failied to export the table information in HTML format. Error: {e}" 

In [ ]:
if __name__ == "__main__":
    print("Export Table Schema Information in HTML format - Started ...")
    errorMessage = _export_table_schema_as_html(db_host, db_port, db_user, db_password, db_name)
    if errorMessage is not None:
        print("Export Table Schema Information in HTML format - Failed")
    print("Export Table Schema Information in HTML format - Ended - Successfully")